# Лабораторна робота №4
# Дровольського Ярослава, ПЗС 1 курс магістратури

**Завдання:** Підібрати оптимальні параметри до мережі (багатошарового персептрону) з lab4.

*Спочатку нагадаємо зміст задачі та суть моделі*

Задача моделі: задача класифікації одягу на датасеті Fashion MNIST.

Всі дані (зображення одягу) в Fashion MNIST поділяються на 10 класів:

0. T-shirt / top (футболка / топ)
1. Trouser (брюки)
2. Pullover (пуловер)
3. Dress (плаття)
4. Coat (пальто)
5. Sandal (сандалі)
6. Shirt (сорочка)
7. Sneaker (кеди)
8. Bag (сумка)
9. Ankle boot (ботильйони).

- **Вхідні значення нейронної мережі:** інтенсивність пікселя (число в діапазоні 0-255), центрована і нормована до інтервалу [-0.5, 0.5]. В зображені 784 (28*28) пікселів. Вхідний шар 784 нейрона.
- **Вихідний шар** 10 нейронів. Кожен нейрон повертає ймовірність (0-1) того, що на зображенні предмет одягу з певної категорії (10 категорій = 10 нейронів)

## Підготовка даних

Імпортуємо необхідні бібліотеки

In [36]:
import numpy as np
import sklearn # algorithms for classical ML
import tensorflow as tf # build and train Neural network
from tensorflow import keras

Завантажимо датасет Fashion MNIST

In [37]:
from keras.datasets import mnist

(x_train, y_train), (x_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()

# x_train, x_val - clothes images (28x28 px) for train and validation sample, correspondingly (sample - вибірка)
# y_train, y_val - correct answers for corresponding images

Центруємо і нормуємо вхідні дані, так, щоб значення змінювалися від `-0.5` до `+0.5`.

In [38]:
x_train_float = x_train.astype(np.float64) / 255 - 0.5
x_val_float = x_val.astype(np.float64) / 255 - 0.5

Перетворимо правильні відповіді `y_train` і `y_val` в one-hot encode.

Тобто у кожного об'єкта з `y_train` та `y_val` створюється 10 нових ознак (кожна відповідає за те, чи належать об'єкт певній категорії). Ознака, яка відповідає за ту категорію, якій належить об'єкт, дорівнює `1`, решта дорівнюють `0`.

In [39]:
y_train_oh = keras.utils.to_categorical(y_train, 10)
y_val_oh = keras.utils.to_categorical(y_val, 10)

## Робота з моделлю

Згідно зі слайдом №6 презентації, **Схема** нашої подальшої роботи для кожного завдання така:
1. перебираємо всі дані в умові значення гіперпараметру
2. при цьому робимо крос-валідацію (перересну перевірку)
3. на основі результатів перехресної перевірки обираємо найкраще значення гіперпараметру

### Базова модель

Для отримання однакових похибок при повторному запуску клітинок навчання моделі (наприклад, під час перевірки викладачем), **фіксуємо** `seed` **рандомності**.


*Пояснення: When re-training a Keras neural network on the same data as before, you’ll rarely get the same results twice. This is due to the fact that neural networks in Keras are using randomness when initializing their weights, so on every run weights are initialized differently, therefore during the learning process these will get updated differently, so the same accuracy results when making predictions are unlikely.* [(Джерело)](https://medium.com/@pop.kristina1/why-loading-a-previously-saved-keras-model-gives-different-results-lessons-learned-aeea1014e0ba)

In [53]:
np.random.seed(100)

import tensorflow
tensorflow.random.set_seed(100)

**Тепер перейдемо до складання моделі й підбору її параметрів**.

На **вхід** будемо подавати картинки, витягнуті у вектор довжини 28 * 28 (= 784).

На **виході** маємо 10 вихідних нейронів за кількістю класів в нашій задачі.

Задаємо ці та описані вище параметри.


Побудуємо **БАЗОВУ МОДЕЛЬ** багатошарового персептрона з двома прихованими шарами по 128 нейронів у кожному. Саме для цієї моделі будемо шукати оптимальні значення параметрів.

На прихованих шарах використовуватимемо функцію elu.

Створимо функцію для побудови та тренування базової моделі

In [70]:
from keras import backend as K
from keras import layers as L


def create_basic_model():
  K.clear_session()

  # create model
  model = keras.Sequential()
  model.add(L.Dense(128, input_dim=784, activation='relu')) # relu(x)=max(0,x)
  model.add(L.Dense(128, activation='elu'))
  model.add(L.Dense(10, activation='softmax'))

  # configure the model with losses and metrics, configure for training
  model.compile(
    loss='categorical_crossentropy', # minimize cross-entropy
    optimizer='adam',
    metrics=['accuracy'] # calculated using validation_data
  )

  return model

'''
How to train:

  # train the model
  model.fit(
    x_train_float.reshape(-1, 28*28),
    y_train_oh,
    batch_size=64, # Number of samples per gradient update.
    epochs=10,
    validation_data=(x_val_float.reshape(-1, 28*28), y_val_oh) # The model will not be trained on this data.
  )

'''


 # Dense is Just regular densely-connected NN layer.
 # first argument of Dense constructor is `units` - Positive integer, dimensionality of the output space.
 # Dense implements the operation: output = activation(dot(input, kernel) + bias)


  # Model documentation: https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
  # Layer documentation: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense


'\nHow to train:\n\n  # train the model\n  model.fit(\n    x_train_float.reshape(-1, 28*28),\n    y_train_oh,\n    batch_size=64, # Number of samples per gradient update. \n    epochs=10,\n    validation_data=(x_val_float.reshape(-1, 28*28), y_val_oh) # The model will not be trained on this data.\n  )\n\n'

Напишемо функцію для виконання крос-валідації моделі

In [71]:
# .fit() calls on already trained models, will train model from xero (ignore previous trained coeffircients, etc.) - not for Keras
# function does not call .fit() on passed model - but call on its copy - https://stackoverflow.com/a/49771618 - not for Keras

from sklearn.model_selection import KFold

# create_model_function - is function to create model. It should return ONLY created and compiled model, NOT trained.
# calculates accuracies for cross-validation
# (X, y) are TRAIN data. It will be divided into local-train and local-validation data on each fold.
def cross_validate(X, y, create_model_function, batch_size, number_of_epochs):
  kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5 folds

  fold_accuracies = []

  for train_index, val_index in kf.split(X):
      X_train, X_val = X[train_index], X[val_index]
      y_train, y_val = y[train_index], y[val_index]

      model = create_model_function()  # Create a new model instance, Keras models need to be re-initialized for each fold.
      model.fit(
          X_train.reshape(-1, 28*28),
          y_train,
          epochs=number_of_epochs,
          batch_size=batch_size,
          verbose=0)  # Train the model

      loss, accuracy = model.evaluate(X_val.reshape(-1, 28*28), y_val, batch_size=batch_size, verbose=1)  # Evaluate on local-validation set

      fold_accuracies.append(accuracy)

  return fold_accuracies

### №1 Підбір кількості нейронів на вхідному шарі
Використайте різну кількість нейронів на вхідному шарі: 400, 600, 800, 1200. Аргументуйте відповідь.

Для цього створимо функцію, яка буде створювати та навчати модель, ідентичну з базовою, але яка *має різну кількість нейронів на вхідному шарі*.

In [73]:
def create_basic_model_with_input_layer_neurons(input_layer_neurons_number):
  K.clear_session()

  # create model
  model = keras.Sequential()
  model.add(L.Dense(input_layer_neurons_number, input_dim=784, activation='relu'))
  model.add(L.Dense(128, activation='elu'))
  model.add(L.Dense(10, activation='softmax'))

  # configure the model with losses and metrics
  model.compile(
    loss='categorical_crossentropy', # minimize cross-entropy
    optimizer='adam',
    metrics=['accuracy'] # calculated using validation_data
  )

  return model

In [76]:
def task_1(number_of_neurons):
  accuracies = cross_validate(
    x_train_float, y_train_oh,
    lambda: create_basic_model_with_input_layer_neurons(number_of_neurons),
    batch_size=64,
    number_of_epochs=10)

  print("\n\n\n\n\n")
  print("accuracies: ", accuracies)
  print("Mean accuracy", accuracies.mean())

**1)** 400 нейронів на вхідному шарі

In [ ]:
task_1(400)

**2)** 600 нейронів на вхідному шарі

In [ ]:
task_1(600)

**3)** 800 нейронів на вхідному шарі

In [ ]:
task_1(800)

**4)** 1200 нейронів на вхідному шарі

In [ ]:
task_1(1200)

Використаємо модель в робочому режимі на деяких валідаційних даних:

**Приклад №1**

In [46]:
img1 = x_val[1]
img1

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  13,  67,   0,
          0,   0,   0,  50,  38,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   8, 120, 209, 226, 247, 237,
        255, 255, 255, 247, 238, 235, 172,  72,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0, 137, 239, 252, 243, 234, 229, 238,
        244, 246, 240, 230, 232, 239, 248, 251, 194,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 102, 255, 231, 228, 227, 228, 233, 230,
        230, 229, 228, 232, 232, 231, 227, 224, 252, 179,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 233, 241, 229, 231, 255, 255, 238, 231,
        227, 238, 246, 228, 230, 227, 234, 235, 229, 241,  20,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 248, 241, 231, 255, 149,  47, 252, 228,
        255, 242, 216, 238, 232, 255, 228, 220, 234, 250,  54,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 255, 240, 232, 255,  15,   0, 255, 237,
        191,   0,   0, 214, 255,  13, 123, 255, 234, 252, 114,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   6, 255, 238, 239, 255, 177,   0, 255, 255,
          0, 130, 116,  47,  65,  43,  37, 255, 236, 249, 162,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  32, 255, 236, 245, 255, 204,   0, 255,  84,
          0,  37,  28,  31,   0,  25,  13, 255, 236, 249, 199,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  53, 255, 236, 250, 250, 231,   2, 255,  21,
          0, 221, 255, 236,  54, 245, 198, 243, 238, 245, 223,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  80, 255, 237, 250, 240, 255,   0,   0,  39,
        157,   0,   0, 215,  94,  20, 126, 255, 237, 239, 250,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 101, 255, 235, 253, 244, 243, 133, 138, 208,
        255, 201, 214, 255, 230,   7, 174, 255, 240, 238, 255,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 126, 255, 233, 255, 248, 233, 255, 255, 240,
        232, 243, 243, 231, 251, 255, 255, 254, 243, 238, 255,   3,   0,
          0,   0],
       [  0,   0,   0,   0, 147, 255, 233, 249, 181, 243, 227, 224, 230,
        234, 230, 230, 235, 228, 235, 222, 207, 255, 236, 255,  35,   0,
          0,   0],
       [  0,   0,   0,   0, 163, 255, 245, 221,  86, 255, 233, 233, 235,
        236, 234, 234, 234, 232, 242, 231, 125, 255, 236, 255,  55,   0,
          0,   0],
       [  0,   0,   0,   0, 181, 254, 255, 200,  69, 255, 228, 232, 234,
        235, 234, 234, 233, 235, 241, 237,  70, 255, 235, 246,  57,   0,
          0,   0],
       [  0,   0,   0,   0, 197, 247, 255, 188, 110, 255, 224, 233, 234,
        234, 234, 234, 234, 234, 240, 253,  69, 255, 236, 248,  77,   0,
          0,   0],
       [  0,   0,   0,   0, 200, 246, 255, 149, 145, 255, 223, 235, 234,
        235, 235, 235, 234, 237, 233, 255,  47, 255, 239, 249,  98,   0,
          0,   0],
       [  0,   0,   0,   0, 204, 243, 255, 111, 173, 255, 227, 235, 235,
        236, 235, 235, 235, 239, 229, 255,  19, 227, 246, 249, 110,   0,
          0,   0],
       [  0,   0,   0,   0, 196, 240, 255, 109, 213, 250, 229, 235, 235,
        236, 235, 237, 236, 237, 226, 255,  55, 203, 251, 245, 120,   0,
          0,   0],
       [  0,   0,   0,   0, 192, 243, 255, 114, 232, 240, 232, 235, 235,
        236, 234, 237, 236, 235, 229, 255, 134, 171, 252, 244, 137,   0,
          0,   0],
       [  0,   0,   0,   0, 189, 251, 255, 154, 238, 233, 236, 234, 235,
        236, 235, 238, 236, 235, 232, 255, 166, 125, 255, 243, 142,   0,
          0,   0],
       [  0,   0,   0,   0, 183, 252, 255, 171, 247, 232, 234, 234, 233,
        233, 232, 234, 233, 234, 233, 240, 223, 128, 255, 242, 151,   0,
          0,   0],
       [  0,   0,   0,   0, 178, 243, 255,  57, 238, 241, 238, 238, 238,
        237, 236, 237, 237, 240, 237, 254, 176,  52, 255, 239, 157,   0,
          0,   0],
       [  0,   0,   0,   0, 188, 240, 25

In [47]:
prediction1 = model.predict(x_val_float.reshape(-1, 28*28)[1:2])
print(prediction1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[[1.0754323e-02 7.2234570e-07 9.8170841e-01 1.1337875e-06 7.0201666e-03
  8.2625441e-08 5.1517563e-04 2.2363762e-09 2.8930799e-10 2.7427308e-10]]


Отримали, що ця річ належить до категорії 2 (пуловер) з ймовірністю `99.95%`. Це відповідає дійсності, бо річ, що зображена на картинці, є пуловером.

**Приклад №2**

In [48]:
img2 = x_val[2]
img2

array([[  0,   0,   0,   0,   0,   0,   0,   0,   1,   0,  67, 177, 129,
        153, 117, 129, 146, 141, 175,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   2,   0, 196, 255, 255,
        255, 241, 251, 255, 245, 255,  76,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 243, 237, 217,
        228, 231, 222, 218, 219, 255, 116,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 235, 231,
        233, 226, 225, 229, 222, 255, 145,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  23, 255, 230, 229,
        233, 229, 225, 227, 221, 241, 167,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  67, 255, 225, 234,
        231, 226, 227, 228, 224, 234, 187,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 114, 249, 217, 235,
        238, 245, 227, 227, 225, 241, 176,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 167, 247, 227, 231,
        250, 200, 248, 231, 224, 251, 147,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 208, 242, 225, 243,
        225,   0, 255, 233, 226, 255, 129,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 223, 238, 223, 255,
        161,   0, 255, 236, 223, 255, 135,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 245, 235, 222, 255,
         72,   0, 248, 254, 217, 255, 145,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 233, 226, 255,
          7,   0, 191, 248, 220, 255, 126,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 233, 231, 251,
          0,   0, 159, 251, 221, 255, 111,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 232, 236, 216,
          0,   0, 131, 253, 220, 255, 111,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 228, 243, 172,
          0,   0,  57, 249, 222, 255, 111,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 230, 244, 139,
          0,   0,  10, 255, 227, 255, 100,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 227, 237, 242, 113,
          0,   0,   6, 255, 230, 255,  89,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 204, 239, 245, 151,
          0,   0,   0, 255, 227, 255,  99,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 151, 245, 238, 198,
          0,   0,   0, 255, 234, 255,  92,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  89, 248, 233, 248,
          0,   0,   0, 252, 237, 255,  90,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  31, 240, 229, 226,
          0,   0,   0, 239, 238, 255,  92,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 226, 233, 237,
         34,   0,   0, 234, 238, 255,  95,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 233, 236, 244,
         68,   0,   0, 209, 241, 237,  98,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   1,   0, 152, 242, 245,
        128,   0,   0, 198, 241, 238,  81,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   

In [49]:
prediction2 = model.predict(x_val_float.reshape(-1, 28*28)[2:3])
print(prediction2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[4.5182727e-10 1.0000000e+00 7.3348723e-14 1.5930256e-08 2.2682888e-10
  4.4047523e-17 1.7106184e-09 3.9186285e-15 8.8719164e-17 3.6053544e-17]]


Отримали, що ця річ належить до категорії 1 (брюки) з ймовірністю `99.9%`. Це відповідає дійсності, бо річ, що зображена на картинці, є брюками.

**Приклад №3**

In [50]:
img3 = x_val[10]
img3

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 145, 190,
         97, 109, 105, 187,  76,   0,   0,   0,   2,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  64, 170, 163, 208,
        243, 240, 229, 173, 162, 154,  18,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  39, 158, 196, 149, 191, 199,
        223, 243, 186, 191, 170, 155, 162, 126,  22,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   3,   0, 105, 161, 150, 141, 130, 199, 239,
        229, 220, 188, 244, 145, 128, 142, 151, 175,  23,   0,   2,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0, 150, 137, 134, 130, 147, 121, 241,
        255, 114, 240, 173, 142, 136, 132, 128, 153, 113,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,  17, 157, 129, 146, 149, 137, 141, 177,
        200, 200, 207, 134, 145, 142, 140, 132, 129, 140,   3,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,  50, 158, 137, 153, 150, 125, 155, 191,
        175, 141, 165, 166, 153, 130, 129, 132, 141, 149,  18,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,  77, 166, 161, 137, 151, 166, 181, 216,
        182, 158, 215, 195, 155, 159, 162, 173, 150, 150,  58,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 104, 159, 170, 140, 132, 182, 198, 208,
        175, 136, 174, 169, 147, 155, 121, 188, 162, 142, 101,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 112, 171, 206, 179, 147, 163, 177, 196,
        192, 146, 187, 153, 142, 144, 136, 245, 202, 132, 137,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 126, 181, 219, 146, 157, 128, 121, 175,
        196, 104, 157, 158, 151, 140, 103,  89, 221, 141, 153,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 157, 175, 216,  55, 175, 153, 151, 188,
        214, 126, 204, 159, 145, 145, 116,  71, 212, 154, 155,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 183, 163, 232,  64, 138, 142, 182, 195,
        200, 158, 231, 146, 130, 161, 132, 105, 225, 146, 157,  10,   0,
          0,   0],
       [  0,   0,   0,   0,   6, 188, 161, 221,  54, 149, 165, 151, 167,
        188, 170, 181, 154, 141, 130, 144,  93, 207, 166, 142,  34,   0,
          0,   0],
       [  0,   0,   0,   0,  42, 182, 165, 236,  29, 146, 170, 141, 166,
        208, 124, 162, 173, 154, 132, 140,  97, 194, 153, 161,  47,   0,
          0,   0],
       [  0,   0,   0,   0,  58, 175, 171, 191,   5, 173, 162, 154, 181,
        207, 100, 161, 179, 155, 146, 149,  76, 192, 162, 150,  62,   0,
          0,   0],
       [  0,   0,   0,   0,  80, 179, 166, 167,  10, 182, 151, 157, 178,
        184, 166, 208, 174, 138, 146, 140,  73, 186, 161, 155,  71,   0,
          0,   0],
       [  0,   0,   0,   0, 100, 165, 171, 138,  51, 194, 157, 133, 187,
        196, 166, 198, 163, 129, 142, 134,  73, 191, 154, 155,  79,   0,
          0,   0],
       [  0,   0,   0,   0, 109, 158, 175, 118,  73, 179, 146, 149, 183,
        207, 141, 184, 171, 153, 137, 147,  73, 195, 157, 154,  85,   0,
          0,   0],
       [  0,   0,   0,   0, 109, 163, 179, 122, 116, 177, 144, 165, 182,
        190, 147, 179, 162, 146, 133, 153,  97, 161, 158, 150,  95,   0,
          0,   0],
       [  0,   0,   0,   0, 107, 158, 167, 163, 105, 137, 161, 163, 203,
        199, 149, 181, 167, 161, 147, 147,  99, 166, 157, 141,  99,   0,
          0,   0],
       [  0,   0,   0,   0, 109, 154, 166, 145,   0,  34,  84,  63, 107,
        149, 132, 153, 129, 124, 149,  56,   0, 114, 159, 141,  97,   0,
          0,   0],
       [  0,   0,   0,   0, 108, 154, 169,  85,   0,   0,   0,   0,   0,
          0,   2,   9,   0,   0,   0,   0,   0,  93, 157, 142,  99,   0,
          0,   0],
       [  0,   0,   0,   0, 101, 153, 169,  80,   0,   3,   0,   1,   0,
          0,   0,   0,   0,   0,   0,   1,   0,  80, 155, 144, 105,   0,
          0,   0],
       [  0,   0,   0,   0, 121, 157, 16

In [51]:
prediction3 = model.predict(x_val_float.reshape(-1, 28*28)[10:11])
print(prediction3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[3.3148459e-05 7.0027031e-06 1.5823666e-02 8.0435717e-07 9.7216570e-01
  4.3759633e-07 1.1968223e-02 9.2737125e-08 6.0377893e-08 8.5606467e-07]]


Отримали, що ця річ належить до категорії 4 (пальто) з ймовірністю `95.77%`. Це відповідає дійсності, бо річ, що зображена на картинці, є пальто.